# Image Processing Basics

This notebook demonstrates image processing with PIL, OpenCV, scikit-image, and imageio.

**Libraries:**
- [Pillow](https://pillow.readthedocs.io/) - Python Imaging Library fork
- [OpenCV](https://docs.opencv.org/) - Computer vision library
- [scikit-image](https://scikit-image.org/) - Image processing in Python
- [imageio](https://imageio.readthedocs.io/) - Reading and writing images

In [ ]:
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance
import cv2
from skimage import io, filters, feature, color, transform, exposure
from skimage.draw import disk, rectangle
import imageio.v3 as iio
import matplotlib.pyplot as plt

%matplotlib inline

print(f"OpenCV version: {cv2.__version__}")

## Create Sample Image

Create a sample image with shapes for demonstration.

In [ ]:
# Create a sample image using PIL
width, height = 400, 300
sample_image = Image.new("RGB", (width, height), color=(255, 255, 255))
draw = ImageDraw.Draw(sample_image)

# Draw shapes
draw.rectangle([50, 50, 150, 150], fill=(255, 0, 0), outline=(0, 0, 0))
draw.ellipse([200, 50, 350, 200], fill=(0, 255, 0), outline=(0, 0, 0))
draw.polygon([(100, 200), (50, 280), (150, 280)], fill=(0, 0, 255), outline=(0, 0, 0))

# Add gradient background
for i in range(width):
    for j in range(height):
        pixel = sample_image.getpixel((i, j))
        if pixel == (255, 255, 255):
            gray = int(255 * (1 - j / height * 0.3))
            sample_image.putpixel((i, j), (gray, gray, int(gray * 0.9)))

print(f"Sample image created")
print(f"Size: {sample_image.size}")
print(f"Mode: {sample_image.mode}")

# Display
plt.figure(figsize=(8, 6))
plt.imshow(sample_image)
plt.title("Sample Image")
plt.axis("off")
plt.show()

## PIL (Pillow) Operations

Basic image operations with Pillow.

In [ ]:
# Basic info
img = sample_image.copy()
print(f"Image size: {img.size}")
print(f"Image mode: {img.mode}")

In [ ]:
# Resize and rotate
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# Original
axes[0].imshow(img)
axes[0].set_title(f"Original ({img.size})")
axes[0].axis("off")

# Resized
resized = img.resize((200, 150), Image.Resampling.LANCZOS)
axes[1].imshow(resized)
axes[1].set_title(f"Resized ({resized.size})")
axes[1].axis("off")

# Rotated
rotated = img.rotate(45, expand=True, fillcolor=(128, 128, 128))
axes[2].imshow(rotated)
axes[2].set_title("Rotated 45°")
axes[2].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Apply filters
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Original
axes[0, 0].imshow(img)
axes[0, 0].set_title("Original")
axes[0, 0].axis("off")

# Gaussian Blur
blurred = img.filter(ImageFilter.GaussianBlur(radius=5))
axes[0, 1].imshow(blurred)
axes[0, 1].set_title("Gaussian Blur")
axes[0, 1].axis("off")

# Edge Detection
edges = img.filter(ImageFilter.FIND_EDGES)
axes[0, 2].imshow(edges)
axes[0, 2].set_title("Edge Detection")
axes[0, 2].axis("off")

# Sharpen
sharpened = img.filter(ImageFilter.SHARPEN)
axes[1, 0].imshow(sharpened)
axes[1, 0].set_title("Sharpened")
axes[1, 0].axis("off")

# Contrast Enhancement
enhancer = ImageEnhance.Contrast(img)
contrast = enhancer.enhance(1.5)
axes[1, 1].imshow(contrast)
axes[1, 1].set_title("Enhanced Contrast")
axes[1, 1].axis("off")

# Grayscale
grayscale = img.convert("L")
axes[1, 2].imshow(grayscale, cmap="gray")
axes[1, 2].set_title("Grayscale")
axes[1, 2].axis("off")

plt.tight_layout()
plt.show()

## OpenCV Operations

OpenCV provides extensive computer vision functionality.

In [ ]:
# Convert PIL image to OpenCV format (RGB to BGR)
cv_img = cv2.cvtColor(np.array(sample_image), cv2.COLOR_RGB2BGR)
print(f"OpenCV image shape: {cv_img.shape}")
print(f"OpenCV image dtype: {cv_img.dtype}")

In [ ]:
# OpenCV operations
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Convert to RGB for display
cv_rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
axes[0, 0].imshow(cv_rgb)
axes[0, 0].set_title("Original")
axes[0, 0].axis("off")

# Gaussian Blur
cv_blurred = cv2.GaussianBlur(cv_img, (15, 15), 0)
axes[0, 1].imshow(cv2.cvtColor(cv_blurred, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title("Gaussian Blur")
axes[0, 1].axis("off")

# Canny Edge Detection
cv_gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
cv_edges = cv2.Canny(cv_gray, 100, 200)
axes[0, 2].imshow(cv_edges, cmap="gray")
axes[0, 2].set_title("Canny Edges")
axes[0, 2].axis("off")

# Binary Thresholding
_, cv_thresh = cv2.threshold(cv_gray, 127, 255, cv2.THRESH_BINARY)
axes[1, 0].imshow(cv_thresh, cmap="gray")
axes[1, 0].set_title("Binary Threshold")
axes[1, 0].axis("off")

# Adaptive Thresholding
cv_adaptive = cv2.adaptiveThreshold(
    cv_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
)
axes[1, 1].imshow(cv_adaptive, cmap="gray")
axes[1, 1].set_title("Adaptive Threshold")
axes[1, 1].axis("off")

# Morphological Operations
kernel = np.ones((5, 5), np.uint8)
cv_dilated = cv2.dilate(cv_edges, kernel, iterations=1)
axes[1, 2].imshow(cv_dilated, cmap="gray")
axes[1, 2].set_title("Dilated Edges")
axes[1, 2].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Drawing with OpenCV
cv_draw = cv_img.copy()

# Draw shapes
cv2.rectangle(cv_draw, (10, 10), (100, 50), (0, 255, 255), 2)
cv2.circle(cv_draw, (300, 150), 40, (255, 0, 255), 3)
cv2.line(cv_draw, (0, 0), (400, 300), (255, 255, 0), 2)
cv2.putText(cv_draw, "OpenCV", (150, 280), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

plt.figure(figsize=(8, 6))
plt.imshow(cv2.cvtColor(cv_draw, cv2.COLOR_BGR2RGB))
plt.title("OpenCV Drawing")
plt.axis("off")
plt.show()

## scikit-image Operations

scikit-image provides scientific image processing algorithms.

In [ ]:
# Convert to numpy array for scikit-image
ski_img = np.array(sample_image)
print(f"scikit-image array shape: {ski_img.shape}")
print(f"scikit-image array dtype: {ski_img.dtype}")

In [ ]:
# scikit-image operations
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# Grayscale
ski_gray = color.rgb2gray(ski_img)
axes[0, 0].imshow(ski_gray, cmap="gray")
axes[0, 0].set_title("Grayscale")
axes[0, 0].axis("off")

# Sobel Edge Detection
ski_sobel = filters.sobel(ski_gray)
axes[0, 1].imshow(ski_sobel, cmap="gray")
axes[0, 1].set_title("Sobel Edges")
axes[0, 1].axis("off")

# Canny Edge Detection
ski_canny = feature.canny(ski_gray, sigma=2)
axes[0, 2].imshow(ski_canny, cmap="gray")
axes[0, 2].set_title("Canny Edges")
axes[0, 2].axis("off")

# Gaussian Filter
ski_gaussian = filters.gaussian(ski_img, sigma=3, channel_axis=-1)
axes[1, 0].imshow(ski_gaussian)
axes[1, 0].set_title("Gaussian Filter")
axes[1, 0].axis("off")

# Histogram Equalization
ski_eq = exposure.equalize_hist(ski_gray)
axes[1, 1].imshow(ski_eq, cmap="gray")
axes[1, 1].set_title("Histogram Equalization")
axes[1, 1].axis("off")

# Contrast Stretching
p2, p98 = np.percentile(ski_gray, (2, 98))
ski_rescale = exposure.rescale_intensity(ski_gray, in_range=(p2, p98))
axes[1, 2].imshow(ski_rescale, cmap="gray")
axes[1, 2].set_title("Contrast Stretching")
axes[1, 2].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Geometric transforms
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# Original
axes[0].imshow(ski_img)
axes[0].set_title("Original")
axes[0].axis("off")

# Resize
ski_resized = transform.resize(ski_img, (150, 200), anti_aliasing=True)
axes[1].imshow(ski_resized)
axes[1].set_title(f"Resized {ski_resized.shape[:2]}")
axes[1].axis("off")

# Rotate
ski_rotated = transform.rotate(ski_img, 30, resize=True)
axes[2].imshow(ski_rotated)
axes[2].set_title("Rotated 30°")
axes[2].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Feature detection - Harris corners
ski_corners = feature.corner_harris(ski_gray)
ski_corners_peaks = feature.corner_peaks(ski_corners, min_distance=10)

print(f"Detected {len(ski_corners_peaks)} corners using Harris detector")

# Visualize corners
plt.figure(figsize=(8, 6))
plt.imshow(ski_img)
plt.scatter(ski_corners_peaks[:, 1], ski_corners_peaks[:, 0], c="red", s=50, marker="+")
plt.title(f"Harris Corner Detection ({len(ski_corners_peaks)} corners)")
plt.axis("off")
plt.show()

## imageio Operations

imageio provides simple I/O for images and videos.

In [ ]:
# imageio reads directly to numpy arrays
iio_img = np.array(sample_image)
print(f"imageio array shape: {iio_img.shape}")
print(f"imageio array dtype: {iio_img.dtype}")

In [ ]:
# Create animated GIF with rotating images
print("Creating animated GIF...")
frames = []
for angle in range(0, 360, 30):
    rotated_frame = transform.rotate(ski_img, angle, resize=False)
    frames.append((rotated_frame * 255).astype(np.uint8))

print(f"Created {len(frames)} frames")

# Display some frames
fig, axes = plt.subplots(1, 4, figsize=(12, 3))
for i, ax in enumerate(axes):
    ax.imshow(frames[i * 3])
    ax.set_title(f"Frame {i*3} ({i*3*30}°)")
    ax.axis("off")
plt.tight_layout()
plt.show()

print("\nTo save as GIF: iio.imwrite('animation.gif', frames, duration=100, loop=0)")

---

## Summary

In this notebook, we covered:

1. **PIL (Pillow)**: Resize, rotate, filters (blur, edges, sharpen), color enhancements
2. **OpenCV**: Gaussian blur, Canny edges, thresholding, morphological operations, drawing
3. **scikit-image**: Sobel/Canny edges, Gaussian filter, histogram equalization, transforms, corner detection
4. **imageio**: Reading/writing images, creating animated GIFs

**Library Comparison:**
| Library | Strengths | Use Case |
|---------|-----------|----------|
| Pillow | Simple API, format support | Basic editing, thumbnails |
| OpenCV | Fast, real-time processing | Computer vision, video |
| scikit-image | Scientific algorithms | Research, analysis |
| imageio | Simple I/O | Reading/writing formats |

For more information:
- [Pillow Documentation](https://pillow.readthedocs.io/)
- [OpenCV-Python Tutorials](https://docs.opencv.org/master/d6/d00/tutorial_py_root.html)
- [scikit-image Gallery](https://scikit-image.org/docs/stable/auto_examples/)